# Using GridSearchCV with Tensorflor: Classification

In [1]:
import tensorflow as tf
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
# Load data
dataObj = load_breast_cancer()
X = dataObj.data
y = dataObj.target

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.30, random_state=1
)

# Standardization
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [3]:
def get_clf_model(hidden_layer_sizes, meta, compile_kwargs):

    model = tf.keras.Sequential()
    input_layer = tf.keras.layers.Input(shape=(meta["n_features_in_"]))
    model.add(input_layer)

    for i, hidden_layer_size in enumerate(hidden_layer_sizes):
        hidden_layer = tf.keras.layers.Dense(
            units=hidden_layer_size, activation="relu", name=f"hidden{i+1}"
        )
        model.add(hidden_layer)

    if meta["target_type_"] == "binary":
        n_output_units = 1
        output_activation = "sigmoid"
        loss = "binary_crossentropy"
    elif meta["target_type_"] == "multiclass":
        n_output_units = meta["n_classes_"]
        output_activation = "softmax"
        loss = "sparse_categorical_crossentropy"
    else:
        raise NotImplementedError(f"Unsupported task type: {meta['target_type_']}")

    output_layer = tf.keras.layers.Dense(n_output_units, activation=output_activation)
    model.add(output_layer)

    model.compile(
        optimizer=compile_kwargs["optimizer"],
        loss=loss,
        metrics=["accuracy"],
    )

    return model

In [4]:
# Note that I am setting validation_split=0, because already use GridSearchCV which has cross-validation.

clf = KerasClassifier(
    model=get_clf_model,
    hidden_layer_sizes=(16, 3),
    optimizer="adam",
    optimizer__learning_rate=0.001,
    batch_size=16,
    epochs=200,
    validation_split=0,
    verbose=1,
)

In [5]:
set1 = {
    "optimizer__learning_rate": [0.01, 0.1],
    "hidden_layer_sizes": [(16, 3), (8, 3)],
}

param_grid = [set1]

In [6]:
gs = GridSearchCV(estimator=clf, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  cv=5,
                  n_jobs=-1)

In [7]:
gs.fit(X_train_std,y_train)

2024-03-01 15:20:47.897199: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-03-01 15:20:47.897202: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-03-01 15:20:47.897235: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-03-01 15:20:47.897211: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-03-01 15:20:47.897236: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-03-01 15:20:47.897246: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-03-01 15:20:47.897243: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-03-01 15:20:47.897256: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-03-01 15:20:47.897259: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-03-01 15:20:47.897232: I metal_plugin/src/device/metal_device.cc:1154] Metal device se

Epoch 1/200
Epoch 1/200
Epoch 1/200
Epoch 1/200
Epoch 1/200
Epoch 1/200
Epoch 1/200
Epoch 1/200


2024-03-01 15:20:49.098703: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-01 15:20:49.098697: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-01 15:20:49.118730: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-01 15:20:49.120314: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-01 15:20:49.123312: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-01 15:20:49.125673: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-01 15:20:49.144039: I tensorflow/core/grappler/optimizers/cust

20/20 [==============================] - 1s 26ms/step - loss: 0.1959 - accuracy: 0.9308
Epoch 2/200
20/20 [==============================] - 2s 28ms/step - loss: 0.2052 - accuracy: 0.8994
Epoch 2/200
20/20 [==============================] - 2s 28ms/step - loss: 0.5737 - accuracy: 0.9025
Epoch 2/200
20/20 [==============================] - 0s 9ms/step - loss: 0.0745 - accuracy: 0.9843
Epoch 3/200
20/20 [==============================] - 0s 9ms/step - loss: 0.0638 - accuracy: 0.9780
Epoch 3/200
20/20 [==============================] - 0s 12ms/step - loss: 0.1757 - accuracy: 0.9654
Epoch 3/200
20/20 [==============================] - 0s 16ms/step - loss: 0.0637 - accuracy: 0.9843
Epoch 4/200
20/20 [==============================] - 0s 16ms/step - loss: 0.0550 - accuracy: 0.9811
Epoch 4/200
20/20 [==============================] - 2s 60ms/step - loss: 0.2013 - accuracy: 0.9245
Epoch 2/200
20/20 [==============================] - 0s 19ms/step - loss: 0.1250 - accuracy: 0.9717
Epoch 4/200
20

2024-03-01 15:21:52.779906: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-01 15:21:53.014265: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


...

2024-03-01 15:23:34.750582: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-01 15:23:34.812821: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-03-01 15:23:34.813399: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-03-01 15:23:34.813409: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-03-01 15:23:34.813484: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-01 15:23:34.813498: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/200


2024-03-01 15:23:35.714941: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


25/25 [==============================] - 3s 35ms/step - loss: 0.1664 - accuracy: 0.9322
Epoch 2/200
25/25 [==============================] - 0s 6ms/step - loss: 0.0778 - accuracy: 0.9799
Epoch 3/200
25/25 [==============================] - 0s 6ms/step - loss: 0.0645 - accuracy: 0.9849
Epoch 4/200
25/25 [==============================] - 0s 6ms/step - loss: 0.0596 - accuracy: 0.9874
Epoch 5/200
25/25 [==============================] - 0s 5ms/step - loss: 0.0626 - accuracy: 0.9849
Epoch 6/200
25/25 [==============================] - 0s 5ms/step - loss: 0.0576 - accuracy: 0.9849
Epoch 7/200
25/25 [==============================] - 0s 6ms/step - loss: 0.0604 - accuracy: 0.9899
Epoch 8/200
25/25 [==============================] - 0s 5ms/step - loss: 0.0588 - accuracy: 0.9874
Epoch 9/200
25/25 [==============================] - 0s 5ms/step - loss: 0.0624 - accuracy: 0.9849
Epoch 10/200
25/25 [==============================] - 0s 5ms/step - loss: 0.0621 - accuracy: 0.9874
Epoch 11/200
25/25 [

GridSearchCV(cv=5,
             estimator=KerasClassifier(batch_size=16, epochs=200, hidden_layer_sizes=(16, 3), model=<function get_clf_model at 0x2b2a1b880>, optimizer='adam', optimizer__learning_rate=0.001, validation_split=0),
             n_jobs=-1,
             param_grid=[{'hidden_layer_sizes': [(16, 3), (8, 3)],
                          'optimizer__learning_rate': [0.01, 0.1]}],
             scoring='accuracy')

In [8]:
df = pd.DataFrame(gs.cv_results_)
df = df.sort_values(by=['rank_test_score'])
display(df.head())

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,param_optimizer__learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,66.210966,1.460546,0.225169,0.045431,"(16, 3)",0.01,"{'hidden_layer_sizes': (16, 3), 'optimizer__le...",0.9875,1.0000,0.9750,0.936709,0.974684,0.974778,0.021201,1
3,40.369041,12.851400,0.072039,0.012727,"(8, 3)",0.1,"{'hidden_layer_sizes': (8, 3), 'optimizer__lea...",0.9500,0.9875,0.9375,0.962025,0.987342,0.964873,0.019977,2
1,65.793714,0.668824,0.296654,0.176250,"(16, 3)",0.1,"{'hidden_layer_sizes': (16, 3), 'optimizer__le...",0.9500,0.9875,0.9625,0.835443,0.987342,0.944557,0.056448,3
2,66.543695,0.312406,0.180108,0.088010,"(8, 3)",0.01,"{'hidden_layer_sizes': (8, 3), 'optimizer__lea...",0.9625,0.8500,0.9500,0.962025,0.987342,0.942373,0.047759,4


In [9]:
gs.best_params_

{'hidden_layer_sizes': (16, 3), 'optimizer__learning_rate': 0.01}

In [10]:
gs.predict(X_test_std)
testing_accuracy = gs.score(X_test_std,y_test)
print(f"Testing accuracy: {testing_accuracy:6.3f}")

 1/11 [=>............................] - ETA: 0s

2024-03-01 15:24:08.617641: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - 0s 1ms/step
Testing accuracy:  0.959
